# Load in Libraries and Setup Plotting Environment

In [1]:
# data loading and manipulation
import os
import requests
import bs4 as bs

import urllib.request
import ssl
import lxml
context = ssl._create_unverified_context()
from unidecode import unidecode

import re
import json
import pandas as pd
import numpy as np

In [2]:
# plotting
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import ListedColormap
%matplotlib inline

# figure aesthetics
sns.set(font_scale=1.5, style='whitegrid')

# Load in and Clean Data

## Load Grape Type Data from Wikipedia

In [3]:
def extract_grape_variety(table_key, parsed_wiki):
    """
    Extract grape varieties from Wikipedia.
    
    Input:
        table_key (string): key to find table for grape (red, white)
        parsed_wiki (BeautifulSoup tree): parsed wiki HTML tree
    
    Output:
        (list): all grapes found in the table.
    """
    # get table and children
    grape_table = parsed_wiki.find(id=table_key).parent.next_sibling.next_sibling
    children = grape_table.findAll('tr')
    
    # loop through all children and get grape text
    output_variety_list = []
    for child in children[1:]:
        curr_grap_variety = child.find('td').text

        # remove any text between parenthesis and brackets
        curr_grap_variety = re.sub(r'\[(.*?)\]', '', curr_grap_variety)
        curr_grap_variety = re.sub(r'\((.*?)\)', '', curr_grap_variety)

        # separate text with '/'
        curr_grap_variety_list = curr_grap_variety.split('/')

        # remove accents, make lowercase, remove any white space
        curr_grap_variety_list = [unidecode(x).lower().strip() for x in curr_grap_variety_list]

        # add to existing grape varietes
        output_variety_list.extend(curr_grap_variety_list)
    
    return output_variety_list

In [4]:
wikipedia_grapes_url = 'https://en.wikipedia.org/wiki/List_of_grape_varieties'
wikipedia_grapes_source = urllib.request.urlopen(wikipedia_grapes_url, context=context).read()
wikipedia_grapes_parsed = bs.BeautifulSoup(wikipedia_grapes_source, 'lxml')

In [5]:
red_grape_varieties = extract_grape_variety('Red_grapes', wikipedia_grapes_parsed)
white_grape_varieties = extract_grape_variety('White_grapes', wikipedia_grapes_parsed)
rose_grape_varieties = extract_grape_variety('Rose_Grapes', wikipedia_grapes_parsed)

In [6]:
red_grape_varieties[0:10]

['abbuoto',
 'abouriou',
 'alexandroouli',
 'akhasheni',
 'abrusco',
 'acolon',
 'ada karasi',
 'affenthaler',
 'agiorgitiko',
 'aglianico']

In [7]:
white_grape_varieties[0:10]

['adalmiina',
 'addoraca',
 'aidani',
 'aidini',
 'aedani',
 'airen',
 'alarije',
 'alarijen',
 'albalonga',
 'albana']

In [8]:
rose_grape_varieties[0:10]

['agdam gyzyl uzumu', 'chardonnay rose']

## Load in TopoJson for Country Labeling
Sourced from: https://gist.github.com/alexwebgr/10249781

In [9]:
with open('../data/world-topo.json') as f:
    topo_json = json.load(f)
countries = topo_json['objects']['countries']['geometries']

In [10]:
def find_code_for_country(country_name, country_topo):
    """
    Retrieves country code given name for country.
    
    Input: 
        country_name (string): name of country to find code for.
        country_topo (list of dicts): country topo containing name and country code.
    
    Output:
        (string): country code or None
    """
    for country in country_topo:
        if country['properties']['name'].lower() == country_name.lower():
            return country['properties']['countryCode']
    
    return None

## Attempt 1: Global Wine Score API

In [11]:
global_wine_score_path = '../data/global_wine_score.csv'
global_wine_score_df = None

# check if data is saved locally, otherwise use API
if os.path.isfile(global_wine_score_path):
    print('Data loaded from file {}'.format(global_wine_score_path))
    global_wine_score_df = pd.read_csv(global_wine_score_path)
else:
    wine_list_json = []

    api_url = 'https://api.globalwinescore.com/globalwinescores/latest/'
    limit = 10000
    offset = 0
    has_next = True

    while has_next:
        # fetch data from global winescore api
        print('Fetching records {} to {} from {}?limit={}&offset={}'.format(offset, offset + limit, api_url, limit, offset))
        response = requests.get(api_url,
                            params={'limit': limit, 'offset': offset},
                            headers={'Authorization': 'Token {}'.format(os.environ['GLOBAL_WINE_SCORE_API'])})

        # add to existing data list
        if response.status_code == 200: 
            # parse json from response
            response_json = response.json()

            # add data to existing list
            wine_list_json.extend(response_json['results'])

            # check if a next is available
            if response_json['next'] is not None:
                offset += limit
            else:
                has_next = False

    global_wine_score_df = pd.DataFrame.from_dict(wine_list_json, orient='columns')
    global_wine_score_df.to_csv(global_wine_score_path, index=False)

# show loaded dataframe
global_wine_score_df.head()

Data loaded from file ../data/global_wine_score.csv


,appellation,appellation_slug,classification,color,confidence_index,country,date,is_primeurs,journalist_count,lwin,lwin_11,regions,score,vintage,wine,wine_id,wine_slug,wine_type
0,Bonnes Mares Grand Cru,bonnes-mares-grand-cru,NaN,Red,A,France,2019-05-31,False,5,1056789.0,1.056789e+10,['Bourgogne'],96.30,2016,"Domaine Georges & Christophe Roumier, Bonnes M...",58794,domaine-georges-christophe-roumier-bonnes-mare...,NaN
1,Puente Alto,puente-alto,NaN,Red,A,Chile,2019-05-31,False,3,1083246.0,1.083246e+10,['Chile'],95.63,2016,"Vina Almaviva, Puente Alto",140620,vina-almaviva-puente-alto,NaN
2,Cote Rotie,cote-rotie,NaN,Red,B+,France,2019-05-31,False,4,1111426.0,1.111426e+10,['Rhone'],95.45,2016,"Delas Freres, La Landonne, Cote Rotie",49900,delas-freres-la-landonne-cote-rotie,NaN
3,Hermitage,hermitage,NaN,Red,A,France,2019-05-31,False,6,1111497.0,1.111497e+10,['Rhone'],95.15,2016,"Delas Freres, Les Bessards, Hermitage",49902,delas-freres-les-bessards-hermitage,NaN
4,Hermitage,hermitage,NaN,Red,B,France,2019-05-31,False,4,NaN,NaN,['Rhone'],93.14,2015,"E. Guigal, Hermitage",68469,e-guigal-hermitage,NaN


## Attempt 2: [Kaggle](https://www.kaggle.com/zynicide/wine-reviews#winemag-data-130k-v2.csv) Wine Review Data

In [12]:
def extract_vintage(wine_title):
    """
    Extracts vintage of a wine from a given wine title. 
    
    Inputs:
        wine_title (string): title for wine possibly containing vintage
    
    Output: 
        (int or None): year as int or None if title has no year
    """
    vintage_list = re.findall(r'\b\d{4}\b', wine_title)
    
    # check if there is one and only one year, and not a champagne
    if len(vintage_list) == 1 and not ' NV ' in wine_title:
        vintage_year = int(float(vintage_list[0]))
        
        # check if newer than 1900
        if vintage_year >= 1900: 
            return vintage_year
    return None

In [13]:
def label_variety_by_type(title, variety, red_wine_grapes, white_wine_grapes, rose_wine_grapes):
    """
    Generates a label (red, white, rose) based on the variety of wine grape.
    
    Input: 
        title (string): name of wine.
        variety (string): variety of grape used for wine.
        red_wine_grapes (list): list of grapes used in red wines.
        white_wine_grapes (list): list of grapes used in white wines.
        rose_wine_grapes (list): list of grapes used in rose wines.
    
    Output:
        (string): label for wine (either red, white, or rose). None if cannot be found.
    """
    # make everything lower case with no accents and whitespace at beginning/end 
    title = unidecode(title).lower().strip()
    variety = unidecode(variety).lower().strip()
    
    # remove "blend"
    variety = re.sub(r'blend', '', variety).strip()
    
    # some varieties have multiple grapes, so split by '-'
    varieties = variety.split('-')
    
    # make wine lists into sets for quick lookup
    red_wine_grapes_set = set(red_wine_grapes)
    white_wine_grapes_set = set(white_wine_grapes)
    rose_wine_grapes_set = set(rose_wine_grapes)
    
    # check if Rose is in the title of the wine
    if 'rose' in title:
        return 'Rose'
    
    # check if white or red is in the variety for blends
    if 'red' in variety:
        return 'Red'
    
    if 'white' in variety:
        return 'White'
    
    # check grape lists contain the varieties
    output_type = None
    for curr_variety in varieties:
        # check current variety type
        curr_variety_type = ''
        
        if curr_variety in red_wine_grapes_set:
            curr_variety_type = 'Red'
        elif curr_variety in white_wine_grapes_set:
            curr_variety_type = 'White'
        elif curr_variety in rose_wine_grapes_set:
            curr_variety_type = 'Rose'
            
        # set output_type if not already set
        if output_type is None and curr_variety_type != '':
            output_type = curr_variety_type
        else:
            # check if all variety types are the same, otherwise None and stop checking
            if output_type != curr_variety_type:
                output_type = None
                break
    
    # return output_type
    return output_type

In [14]:
raw_data_path = '../data/'
wine_130k_file = 'winemag-data-130k-v2.csv'

In [15]:
# convert csv to pandas df
wine_130k_df = pd.read_csv(raw_data_path + wine_130k_file, index_col=0)

# remove unneeded columns for taster
del wine_130k_df['taster_name'], wine_130k_df['taster_twitter_handle']

# add a vintage column
wine_130k_df = wine_130k_df.assign(vintage=wine_130k_df.title.apply(extract_vintage))

# remove any and all NAs
wine_130k_df.dropna(subset=['country', 'points', 'price', 'province', 'title', 'variety', 'winery', 'vintage'], inplace=True)

# make points and vintage column int
wine_130k_df['points'] = pd.to_numeric(wine_130k_df['points'])
wine_130k_df['vintage'] = wine_130k_df['vintage'].astype(int)

# replace bad countries for code replacement
wine_130k_df.replace({
    'US': 'United States',
    'England': 'United Kingdom',
    'Moldova': 'Moldova, Republic of',
    'Macedonia': 'Macedonia, the former Yugoslav Republic of'
}, inplace=True)

# add country codes and remove any blank ones
wine_130k_df = wine_130k_df.assign(code=wine_130k_df.country.apply(lambda x: find_code_for_country(x, countries)))
wine_130k_df.dropna(subset=['code'], inplace=True)

# add wine color column
wine_130k_df = wine_130k_df.assign(color=wine_130k_df.apply(lambda x: label_variety_by_type(x['title'],
                                                                                            x['variety'],
                                                                                            red_grape_varieties,
                                                                                            white_grape_varieties,
                                                                                            rose_grape_varieties), axis=1))

# remove all None wine colors
wine_130k_df.dropna(subset=['color'], inplace=True)

# reset index 
wine_130k_df.reset_index(drop=True, inplace=True)

# save out full data and sample of 1000 wines
wine_130k_df.to_json('../data/wine_reviews.json')

wine_1k_df = wine_130k_df.sample(1000, random_state=42).reset_index(drop=True)
wine_1k_df.to_json('../data/wine_reviews_small.json', orient='records')

In [16]:
print('Number of records: {}'.format(len(wine_130k_df)))
wine_130k_df.head()

Number of records: 110555


,country,description,designation,points,price,province,region_1,region_2,title,variety,winery,vintage,code,color
0,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,PT,Red
1,United States,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,US,White
2,United States,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,US,White
3,United States,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,US,Red
4,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011,ES,Red


In [17]:
wine_130k_df.describe(include='all')

,country,description,designation,points,price,province,region_1,region_2,title,variety,winery,vintage,code,color
count,110555,110555,77660,110555.000000,110555.000000,110555,92340,48221,110555,110555,110555,110555.000000,110555,110555
unique,41,101935,32152,NaN,NaN,397,1164,17,101619,488,15047,NaN,40,3
top,United States,"Cigar box, café au lait, and dried tobacco aro...",Reserve,NaN,NaN,California,Napa Valley,Central Coast,Château Lestage Simon 2012 Haut-Médoc,Pinot Noir,Testarossa,NaN,US,Red
freq,51859,3,1932,NaN,NaN,34497,4291,10615,4,12631,216,NaN,51859,69323
mean,NaN,NaN,NaN,88.430157,35.148351,NaN,NaN,NaN,NaN,NaN,NaN,2010.814653,NaN,NaN
std,NaN,NaN,NaN,3.047328,40.937525,NaN,NaN,NaN,NaN,NaN,NaN,3.544005,NaN,NaN
min,NaN,NaN,NaN,80.000000,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,1934.000000,NaN,NaN
25%,NaN,NaN,NaN,86.000000,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,2009.000000,NaN,NaN
50%,NaN,NaN,NaN,88.000000,25.000000,NaN,NaN,NaN,NaN,NaN,NaN,2012.000000,NaN,NaN
75%,NaN,NaN,NaN,91.000000,42.000000,NaN,NaN,NaN,NaN,NaN,NaN,2013.000000,NaN,NaN


In [18]:
print('Number of records: {}'.format(len(wine_1k_df)))
wine_1k_df.head()

Number of records: 1000


,country,description,designation,points,price,province,region_1,region_2,title,variety,winery,vintage,code,color
0,United States,A smokiness rises to the fore initially in thi...,Owsley Vineyard,87,46.0,California,Russian River Valley,Sonoma,Sonoma-Cutrer 2012 Owsley Vineyard Pinot Noir ...,Pinot Noir,Sonoma-Cutrer,2012,US,Red
1,United States,Well differentiated from the winery's Pinot Bl...,NaN,91,20.0,Oregon,Willamette Valley,Willamette Valley,The Four Graces 2015 Pinot Gris (Willamette Va...,Pinot Gris,The Four Graces,2015,US,White
2,United States,"From a high perch on Howell Mountain, this pro...",Estate Vineyards,90,48.0,California,Napa Valley,Napa,Burgess 2012 Estate Vineyards Cabernet Sauvign...,Cabernet Sauvignon,Burgess,2012,US,Red
3,France,"A fruity, crisp and herbaceous blend of Loin d...",Terrisses Sec,87,14.0,Southwest France,Gaillac,NaN,Domaine des Terrisses 2014 Terrisses Sec White...,White Blend,Domaine des Terrisses,2014,FR,White
4,France,"Intriguing, thought provoking and complex, thi...",NaN,90,16.0,Rhône Valley,Côtes du Rhône,NaN,Domaine St Pierre 2014 Red (Côtes du Rhône),Rhône-style Red Blend,Domaine St Pierre,2014,FR,Red


In [19]:
wine_1k_df.describe(include='all')

,country,description,designation,points,price,province,region_1,region_2,title,variety,winery,vintage,code,color
count,1000,1000,694,1000.000000,1000.000000,1000,838,436,1000,1000,1000,1000.000000,1000,1000
unique,23,999,609,NaN,NaN,113,268,17,999,115,897,NaN,23,3
top,United States,Another successful release from Stromberg afte...,Estate,NaN,NaN,California,Napa Valley,Central Coast,Stromberg Vineyards 2006 Cabernet Sauvignon (D...,Pinot Noir,Recanati,NaN,US,Red
freq,463,2,21,NaN,NaN,302,32,85,2,115,4,NaN,463,621
mean,NaN,NaN,NaN,88.714000,35.942000,NaN,NaN,NaN,NaN,NaN,NaN,2011.100000,NaN,NaN
std,NaN,NaN,NaN,2.965974,35.803219,NaN,NaN,NaN,NaN,NaN,NaN,3.434792,NaN,NaN
min,NaN,NaN,NaN,80.000000,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,1997.000000,NaN,NaN
25%,NaN,NaN,NaN,87.000000,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,2009.000000,NaN,NaN
50%,NaN,NaN,NaN,89.000000,26.000000,NaN,NaN,NaN,NaN,NaN,NaN,2012.000000,NaN,NaN
75%,NaN,NaN,NaN,91.000000,43.000000,NaN,NaN,NaN,NaN,NaN,NaN,2014.000000,NaN,NaN


In [20]:
wine_1k_df.country.unique()

array(['United States', 'France', 'Spain', 'Austria', 'Argentina',
       'Italy', 'New Zealand', 'Portugal', 'Chile', 'South Africa',
       'Australia', 'Canada', 'Germany', 'Israel', 'Greece', 'Morocco',
       'Hungary', 'Moldova, Republic of', 'Mexico', 'Slovenia', 'Georgia',
       'Romania', 'Uruguay'], dtype=object)

### Aggregate stats by country

In [21]:
# get aggregated stats by relevant col
wine_1k_agg_df = wine_1k_df[['country', 'code', 'points', 'price', 'color']]
wine_1k_agg_df = wine_1k_agg_df.groupby(['country', 'code']).agg({
    'code': 'count',
    'points': 'mean',
    'price': 'mean'
}).add_suffix('_').reset_index()

In [22]:
# add in count by wine color
wine_color_agg = wine_1k_df.groupby(['country', 'color'])['color'].count().reset_index(name='count').pivot(index='country',columns='color')
wine_color_agg.fillna(0, inplace=True)

wine_1k_agg_df = wine_1k_agg_df.merge(wine_color_agg, on='country')

/Users/kapilgarg/.local/share/virtualenvs/eecs-496-info-viz-a3-BDwxcSbX/lib/python3.7/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [23]:
wine_1k_agg_df.rename(index=str, columns={
    'code_': 'count_total',
    ('count', 'Red'): 'count_red',
    ('count', 'Rose'): 'count_rose',
    ('count', 'White'): 'count_white',
    'country': 'country_name',
    'code': 'country_code',
    'points_': 'avg_points',
    'price_': 'avg_price'
}, inplace=True)

In [24]:
wine_1k_agg_df.head()

,country_name,country_code,count_total,avg_points,avg_price,count_red,count_rose,count_white
0,Argentina,AR,27,87.222222,32.888889,22.0,0.0,5.0
1,Australia,AU,17,89.411765,26.352941,5.0,1.0,11.0
2,Austria,AT,39,90.487179,30.641026,8.0,2.0,29.0
3,Canada,CA,4,89.750000,43.250000,3.0,0.0,1.0
4,Chile,CL,30,86.300000,17.466667,21.0,0.0,9.0


In [25]:
wine_1k_agg_df.to_json('../data/wine_reviews_small_agg.json', orient='records')